## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-block alert-warning">
Этапы проекта

1. [Общая информация о датасете](#step1)
2. [Предобработка данных](#step2)
3. [Ответы на вопросы](#step3)
4. [Общий вывод](#step4)

</div>

### 1. Общая информация о датасете <a id="step1"></a>

Прочитаем файл, сохраним его в переменной data и посмотрим общую информацию

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Откроем первые 10 строк таблицы data для понимания вида датасета

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В результате выполненных шагов были получены следующие выводы:
1. Файл содержит 12 столбцов и 21525 строк. Среди столбцов 7 имеют численный тип (float64 и int64), 5 - строки (object);
2. Столбцы days_employed и total_income имеют меньшее количество строк - 19351. Вероятно, это связано со статусом занятости и, как следствие, отсутствием дохода, поэтому строки были пропущены при их заполнении;
3. Данные в строках необходимо привести к общему виду. Так, в столбце education формат записи одинаковых признаков отличается, что может затруднить дальнейшую работу с датасетом;
4. Значнеия в  days_employed имеют как положительные, так и отрицательные нецелые значения, что противоречит логике названия столбца;
5. Вероятно, знчения в столбцах education_id, family_status_id и debt необходимо сменить на bool.

### 2. Предобработка данных <a id="step2"></a>

### Обработка пропусков

Проверим датасет на наличие пропусков с помощью метода isnull 

In [4]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как видно, в некоторых столбцах days_employed и total_income отсутствует информация. Количество пустых строк в этих столбцах совпадает. Проверим, есть ли между ними взаимосвязь.

In [5]:
data[data['days_employed'].isnull() == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Между столбцами действительно есть зависимость, что говорит о том, что заполняющий анкету или не хотел указывать свои данные, или не является трудовым занятым. Тем не менее, таким образом были заполнены 2174 строки, что составляет 10% от исходной таблицы данных. Такое количество информации лучше не отбрасывать и сохранить. Сгруппируем данные по столбцу income_type, установим медианные значения для каждой группы, которые используем для заполнения пропусков. 

In [6]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Перед началом работы с данными, необходимо их обработать для логического соответствия названию столбцов. Так, в столбце days_employed значения имеют ряд странностей. Во-первых, встречаются как положительные, так и отрицательные числа. Во-вторых, значения слишком большие для количества дней, что говорит о том, что здесь приведены более частотные единицы измерения - часы/минуты/секунды - из чего более вероятным являются часы. Преобразуем столбец в соответствии с данными особенностями и выведим количественные характеристики в столбцах days_employed и total_income для каждой группы.

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data['days_employed'] = data['days_employed'] / 24

In [9]:
data_grouped_days = data.groupby('income_type').agg({'days_employed': ['min', 'max', 'mean', 'median']})
data_grouped_days

days_employed                                          
                          min           max          mean        median
income_type                                                            
безработный      14063.519451  16470.951611  15267.235531  15267.235531
в декрете          137.364998    137.364998    137.364998    137.364998
госслужащий          1.664757    633.043008    141.662371    112.057015
компаньон            1.258139    733.981803     87.980183     64.474259
пенсионер        13697.030025  16739.808353  15208.478802  15217.221094
предприниматель     21.702003     21.702003     21.702003     21.702003
сотрудник            1.005901    766.206246     96.937467     65.591784
студент             24.114648     24.114648     24.114648     24.114648

In [10]:
data_grouped_total = data.groupby('income_type').agg({'total_income': ['min', 'max', 'mean', 'median']})
data_grouped_total

total_income                                            
                           min           max           mean         median
income_type                                                               
безработный       59956.991984  2.027225e+05  131339.751676  131339.751676
в декрете         53829.130729  5.382913e+04   53829.130729   53829.130729
госслужащий       29200.077193  9.104515e+05  170898.309923  150447.935283
компаньон         28702.812889  2.265604e+06  202417.461462  172357.950966
пенсионер         20667.263793  7.351033e+05  137127.465690  118514.486412
предприниматель  499163.144947  4.991631e+05  499163.144947  499163.144947
сотрудник         21367.648356  1.726276e+06  161380.260488  142594.396847
студент           98201.625314  9.820163e+04   98201.625314   98201.625314

Используя полученные данные, заполним пропуски в столбцах. Для заполнения пропусков воспользуемся медианой, так как видно из двух таблиц, что значения распределены неравномерно

In [11]:
data.loc[data['income_type'] == 'сотрудник', 'days_employed'] = data.loc[data['income_type'] == 'сотрудник', 'days_employed'].fillna(65.591784)

data.loc[data['income_type'] == 'компаньон', 'days_employed'] = data.loc[data['income_type'] == 'компаньон', 'days_employed'].fillna(64.474259)

data.loc[data['income_type'] == 'пенсионер', 'days_employed'] = data.loc[data['income_type'] == 'пенсионер', 'days_employed'].fillna(15217.221094)

data.loc[data['income_type'] == 'госслужащий', 'days_employed'] = data.loc[data['income_type'] == 'госслужащий', 'days_employed'].fillna(112.057015)

data.loc[data['income_type'] == 'предприниматель', 'days_employed'] = data.loc[data['income_type'] == 'предприниматель', 'days_employed'].fillna(21.702003)

In [12]:
data.loc[data['income_type'] == 'сотрудник', 'total_income'] = data.loc[data['income_type'] == 'сотрудник', 'total_income'].fillna(142594.396847)

data.loc[data['income_type'] == 'компаньон', 'total_income'] = data.loc[data['income_type'] == 'компаньон', 'total_income'].fillna(172357.950966)

data.loc[data['income_type'] == 'пенсионер', 'total_income'] = data.loc[data['income_type'] == 'пенсионер', 'total_income'].fillna(118514.486412)

data.loc[data['income_type'] == 'госслужащий', 'total_income'] = data.loc[data['income_type'] == 'госслужащий', 'total_income'].fillna(150447.935283)

data.loc[data['income_type'] == 'предприниматель', 'total_income'] = data.loc[data['income_type'] == 'предприниматель', 'total_income'].fillna(499163.144947)

In [13]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски в столбцах days_employed и income_type заполнены. Теперь проверим корректность заполнения других столбцов - children и dob_years

In [14]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце children видны артефакты - значения -1 и 20. Отрицательное значение противоречит логическом смыслу столбца. Значение 20 также является аномальным, так как большинство значений находится в интервале от 0 до 5. Вероятно, данные артефакты связаны с опечатками при заполнении анкет, которые необходимо исправить

In [15]:
data.loc[data['children']== 20,'children'] = 2
data.loc[data['children']== -1,'children'] = 1

In [16]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Теперь рассмотрим столбец dob_years и распределение количества анкет по возрастам

In [17]:
data.groupby('dob_years')['dob_years'].count().sort_values(ascending=False)

dob_years
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Среди значения выделяется 101 строка с возрастом 0. Данные значения заменим медианой по dob_years

In [18]:
dob_median = data['dob_years'].median()
data.loc[data['dob_years']== 0,'dob_years'] = dob_median

In [19]:
data.groupby('dob_years')['dob_years'].count().sort_values(ascending=False)

dob_years
42.0    698
35.0    617
40.0    609
41.0    607
34.0    603
38.0    598
33.0    581
39.0    573
31.0    560
36.0    555
44.0    547
29.0    545
30.0    540
48.0    538
37.0    537
50.0    514
43.0    513
32.0    510
49.0    508
28.0    503
45.0    497
27.0    493
56.0    487
52.0    484
47.0    480
54.0    479
46.0    475
58.0    461
57.0    460
53.0    459
51.0    448
59.0    444
55.0    443
26.0    408
60.0    377
25.0    357
61.0    355
62.0    352
63.0    269
64.0    265
24.0    264
23.0    254
65.0    194
66.0    183
22.0    183
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

### Вывод

Исходная таблица содержала ряд пропусков в столбах days_employed и total_income, которые были заменены на медианные значения по группам income_type. В столбах children и dob_years артефактные значения были скорректированы с помощью метода replace

### Замена типа данных

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


В списке столбцов все типа данных соответствуют их содержанию, кроме days_employed. Для логического соответствия заменим в данном столбце тип данных с float64 на int64, чтобы в столбце содержалась информация о целом количестве рабочих дней

In [21]:
data['days_employed'] = data['days_employed'].astype('int')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

После изучения исходных типов данных датасета, был изменен тип данных в столбце days_employed с float64 на int64 для соответствия названию стобца и единице измерения

### Обработка дубликатов

Прежде чем выполнить проверку строк на наличие дубликатов, приведем к нижнему регистру значения в столбце education

In [22]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [23]:
data.loc[:, 'education'] = data.loc[:, 'education'].str.lower()

In [24]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь, когда даные в education приведены к общему виду, удалим дубликаты в таблице с помощью метода drop_duplicates

In [25]:
data.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,188,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21449,0,14330,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21450,1,88,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21451,3,129,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

С помощтю метода str.lower были приведены к общему виду данные в столбце education. После чего, с помощью метода drop_duplicates были удалены 72 строки-дубликаты в датасете

### Лемматизация

Перед началом лемматизации списков в purpose рассмотрим уникальные значения в этом столбце

In [26]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В полученном списке уникальных значений можно выделить 4 группы назначений кредита: автомобиль, образование, свадьба и недвижимость/жилье. 
Составим функцию, которая определит отношение строки к одной из выделенных групп

In [27]:
from pymystem3 import Mystem
m = Mystem()

def purpose_grouped(row):
    row_str = str(row)
    lemmas = m.lemmatize(row_str)
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'жилье' in lemmas:
        return 'недвижимость'

data['purpose_group'] = data['purpose'].apply(purpose_grouped)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,351,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,167,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,234,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость
3,3,171,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,38,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость
6,0,119,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость
7,0,6,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование
8,2,288,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,91,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость


Теперь, с помощью метода value_counts можно увидеть распределение строк по сгруппированым целям кредита

In [28]:
data['purpose_group'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_group, dtype: int64

### Вывод

Исходный датасет был успешно лемматизирован, строки разделены на группы по назначению кредита и подведен итог, согласно которому 50,52% заявок были для получения кредита на недвижимость, 20,11% на автомобиль, 18,74% на образование и 10,94% на свадьбу

### Категоризация данных

Для того чтобы классифицировать данные в соответствии с целями проекта, необходимо выделить целевые признаки из поставленных вопросов. 
1. "Есть ли зависимость между наличием детей и возвратом кредита в срок?" Для ответа на данный вопрос требуется сгруппировать столбец children;
2. "Есть ли зависимость между семейным положением и возвратом кредита в срок?" В данном случае сгруппируем столбец family_status;
3. "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" Потребуется классифицировать total_income;
4. "Как разные цели кредита влияют на его возврат в срок?" Для ответа на этот вопрос были сгруппированы значения из purpose

Сгруппируем данные в children по количеству детей

In [29]:
def children_grouped(row):
    if row == 0:
        return 'нет детей'
    if row == 1:
        return 'один ребенок'
    if row == 2:
        return 'два ребенка'
    return 'многодетный'
data['children_group'] = data['children'].apply(children_grouped)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_group
0,1,351,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,один ребенок
1,1,167,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,один ребенок
2,0,234,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,нет детей
3,3,171,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,многодетный
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,нет детей


Значения из столбца children сгруппированы в children_group. Выполним аналогичную операцию с family_status

In [30]:
def status_grouped(row):
    if row == 'женат / замужем':
        return 'есть супруг(а)'
    if row == 'гражданский брак':
        return 'есть супруг(а)'
    return 'нет супруги/супруга'
data['status_group'] = data['family_status'].apply(status_grouped)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_group,status_group
0,1,351,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,один ребенок,есть супруг(а)
1,1,167,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,один ребенок,есть супруг(а)
2,0,234,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,нет детей,есть супруг(а)
3,3,171,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,многодетный,есть супруг(а)
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,нет детей,есть супруг(а)


Значения из столбца family_status сгруппированы в две категории - есть супруг(а) и нет супруги/супруга. Теперь перейдем к total_income

In [31]:
def total_income_grouped(row):
    if row <= 50000:
        return '0 - 50 000'
    if row > 50000 and row <= 100000:
        return '50 001 - 100 000'
    if row > 100000 and row <= 150000:
        return '100 001 - 150 000'
    return 'более 150 000'
data['total_income_group'] = data['total_income'].apply(total_income_grouped)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_group,status_group,total_income_group
0,1,351,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,один ребенок,есть супруг(а),более 150 000
1,1,167,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,один ребенок,есть супруг(а),100 001 - 150 000
2,0,234,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,нет детей,есть супруг(а),100 001 - 150 000
3,3,171,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,многодетный,есть супруг(а),более 150 000
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,нет детей,есть супруг(а),более 150 000


Данные по доходам сгруппированы в четыре группы: до 50 000, от 50 000 до 100 000, от 100 000 до 150 000 и свыше 150 000

### Вывод

Исходные данные таблицы были сгруппированы в соответствии с поставленными вопросами. Словаь не был применен, так как нет необходимости в выделении данных в отдельные таблицы и применении идентификаторов

### 3. Ответы на вопросы <a id="step3"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Составим сводную таблицу, содержащую в себе информацию о наличии долгов в зависимости от количества детей

In [32]:
children_pivot = data.pivot_table(index='children_group', columns='debt', values='gender', aggfunc='count')
children_pivot

debt,0,1
children_group,,
два ребенка,1929,202
многодетный,349,31
нет детей,13086,1063
один ребенок,4420,445


Теперь, для ответа на поставленный вопрос, добавим столбец debt_share, в котором будет указана доля людей с долгом в определенной группе по количеству детей

In [33]:
children_count = data['children_group'].value_counts()
children_pivot['debt_share'] = children_pivot[1] / children_count
children_pivot

debt,0,1,debt_share
children_group,,,
два ребенка,1929,202,0.094791
многодетный,349,31,0.081579
нет детей,13086,1063,0.075129
один ребенок,4420,445,0.091470


### Вывод

Согласно полученным результатам, существует зависимость между количеством детей и наличием долга. Так, у тех, кто не имеет детей, самая низкая доля - 0.075. При этом, в группе людей с одним или двумя детьми доля с задолженностями составляет 0.091 и 0.094 соответственно

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Чтобы установить налачие или отсутствие зависимости между семейным положениеми и возвратом кредита в срок, выполним аналогичные действия

In [34]:
status_pivot = data.pivot_table(index='status_group', columns='debt', values='gender', aggfunc='count')
status_pivot

debt,0,1
status_group,,
есть супруг(а),15238,1319
нет супруги/супруга,4546,422


In [35]:
status_count = data['status_group'].value_counts()
status_pivot['debt_share'] = status_pivot[1] / status_count
status_pivot

debt,0,1,debt_share
status_group,,,
есть супруг(а),15238,1319,0.079664
нет супруги/супруга,4546,422,0.084944


### Вывод

Как видно из результатов, существует разница в доле людей с задолженностями в зависимости от семейного положения. В группе, где у человека есть супруг(а), доля с наличием долгов составила 0.079, когда в другой группе этот уровень составил 0.084

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Также рассмотрим зависимость между уровнем дохода и наличием долгов

In [36]:
total_income_pivot = data.pivot_table(index='total_income_group', columns='debt', values='gender', aggfunc='count')
total_income_pivot

debt,0,1
total_income_group,,
0 - 50 000,349,23
100 001 - 150 000,6598,624
50 001 - 100 000,3760,331
более 150 000,9077,763


In [37]:
total_income_count = data['total_income_group'].value_counts()
total_income_pivot['debt_share'] = total_income_pivot[1] / total_income_count
total_income_pivot

debt,0,1,debt_share
total_income_group,,,
0 - 50 000,349,23,0.061828
100 001 - 150 000,6598,624,0.086403
50 001 - 100 000,3760,331,0.080909
более 150 000,9077,763,0.077541


### Вывод

В установленных группах видна зависимость между ежемесячным доходом и наличием долга. Самая маленькая доля у людей с доходом до 50 000 - 0.061, когда самая большая у людей с доходом от 100 000 до 150 000 - 0.086. С дальнейшим ростом ежемесячного дохода доля людей с долгом постепенно снижается и составляет 0.077 в группе с доходом свыше 150 000

- Как разные цели кредита влияют на его возврат в срок?

Для ответа на поставленный вопрос используем классификацию целей кредита, полученную в результате лемматизации 

In [38]:
purpose_group_pivot = data.pivot_table(index='purpose_group', columns='debt', values='gender', aggfunc='count')
purpose_group_pivot

debt,0,1
purpose_group,,
автомобиль,3912,403
недвижимость,10058,782
образование,3652,370
свадьба,2162,186


In [39]:
purpose_group_count = data['purpose_group'].value_counts()
purpose_group_pivot['debt_share'] = purpose_group_pivot[1] / purpose_group_count
purpose_group_pivot

debt,0,1,debt_share
purpose_group,,,
автомобиль,3912,403,0.093395
недвижимость,10058,782,0.072140
образование,3652,370,0.091994
свадьба,2162,186,0.079216


### Вывод

Основываясь на итоговой таблице, можно сказать, что уровень задолженности отличается в зависимости от предполагаемых целей кредита. Самый низкий уровень долга установлен у людей, которые планируют вложить средства в недвижимость - 0.072. Рядом находится группа, которая планирует использовать кредит для свадьбы - 0.079. Две оставшиеся группы - автомобиль и образование - показали более высокий уровень долга - 0.093 и 0.091 соответственно, что указывает на наличие зависимости между целями кредита и вероятностью возврата в срок

### 4. Общий вывод <a id="step4"></a>

В рамках данного проекта:
1. Проанализирован исходный датасет;
2. Заполнены пропуски в days_employed и total_income; 
3. Удалены дубликаты в таблице;
4. Выполнена лемматизация purpose и выделены группы по целям кредитования;
5. Значения в столбцах children, family_status, total_income классифицированы для получения выводов;
6. Составлены сводные таблицы для ответа на поставленные вопросы;
7. Выявлена взаимосвязь между наличием детей и вероятностью возврата кредита в срок;
8. Выявлена взаимосвязь между семейным положением и вероятностью возврата кредита в срок;
9. Выявлена взаимосвязь между уровнем ежемесячного дохода и вероятностью возврата кредита в срок;
10. Выявлена взаимосвязь между целью кредита и вероятностью возврата кредита в срок.